## A framework to compare different formulations over multiple instances with Decision Optimization for DSX

This notebook exploits [`dd_scenario`](https://ibmdecisionoptimization.github.io/dd-scenario-api-doc/) APIs to solve different formulations of the Unit Commitment over multiple input instances of the problem. 

In particular, the scenarios of the DSX DO Model called *uc-models-comparison* (part of this project) are iteratively updated, in order to process results coming from different optimization runs. Such results are used to generate the charts available in the dashboard of the addressed DSX DO Model. Moreover, additional output data are collected and stored in a CSV file, which is consumed by the notebook *display-comparison-kpis* of this project. Finally, that notebook finalizes the model comparison by generating and displaying charts from the summary results.

The following picture illustrates the procedure (loop) implemented in this notebook.

<img src="../do_scenario_framework.png" width="25%">

In [1]:
from dd_scenario import *

import os, glob
import pandas as pd
from IPython.display import display

### Intializing dd_scenario client
client = Client()
mb = client.get_model_builder(name="uc-models-comparison")

### Loading inputs
opt_input_list = glob.glob("{}/datasets/df_*.csv".format(os.environ['DSX_PROJECT_DIR']))
input_dict = {}
for opt_input in opt_input_list:
    df = pd.read_csv(opt_input)
    df_name = os.path.basename(opt_input).split('.')[0]
    print("Loading {}".format(df_name))
    input_dict[df_name] = df


/opt/conda/lib/python2.7/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.3) doesn't match a supported version!
  RequestsDependencyWarning)


Loading df_heatmap_comp
Loading df_load
Loading df_renewable_forecast
Loading df_cost_comp
Loading df_dashboard
Loading df_ren_comp
Loading df_renewable_forecast_aggr
Loading df_renewable_units
Loading df_renewable_units_conf
Loading df_scenarios
Loading df_system_config
Loading df_thermal_units
Loading df_thermal_units_conf
Loading df_scenarios_conf


In [2]:
### This function extracts an instance of the Unit Commitment by providing a target time horizon for the optimization problem 

from datetime import datetime, timedelta
import collections

DateTime = collections.namedtuple('DateTime', 'date time')

def extract_df_by_period(df, start_datetime=DateTime('01/07/2017', '1'), n_days=7):
    start_dt = datetime.strptime("{}_{}:59".format(start_datetime.date, int(start_datetime.time) - 1), '%d/%m/%Y_%H:%M')
    end_date = datetime.strptime(start_datetime.date, '%d/%m/%Y') + timedelta(days=(n_days - 1))
    end_dt = datetime.strptime("{}_23:59".format(end_date.date().strftime('%d/%m/%Y')), '%d/%m/%Y_%H:%M')
    if 'date_obj' not in df.columns:
        df['date_obj'] = df.apply(lambda x: datetime.strptime("{}_{}:59".format(x.Date, x.Hour - 1), '%d/%m/%Y_%H:%M'), axis=1)
    df_new = df.loc[(df['date_obj'] >= start_dt) & (df['date_obj'] <= end_dt)]
    return df_new.drop('date_obj', axis=1)


In [3]:
### The comparison takes place over 7 daily instances of the problem

DateTime = collections.namedtuple('DateTime', 'date time')

curr_dt = '14/06/2018'
last_dt = '20/06/2018'

uc_days = 1

curr_dt_obj = datetime.strptime(curr_dt, '%d/%m/%Y')
last_dt_obj = datetime.strptime(last_dt, '%d/%m/%Y')
its = int((last_dt_obj.date() - curr_dt_obj.date()).days) + 1
#its = 1

In [4]:
### This procedure updates and solves iteratively the three formulations (D - R - S) of the Unit Commitment for the selected instances. 
### A DO Model scenario is created for each formulation of the problem, and it is updated when each instance is solved.
### The overall optimization results are collected in a dataframe that is updated at the end of each optimization run.

table_list = input_dict.keys()
df_final = pd.DataFrame()

for model in ['D', 'R', 'S']:
    scen_name = "Scen-{}".format(model)
    config = {'parameter': ['model'], 'value': ['D']} if model == 'D' else \
                ({'parameter': ['model', 'criteria'], 'value': ['S', 'RiskFactor']} if model == 'S' else \
                 {'parameter': ['model'], 'value': ['R']})
    df_config = pd.DataFrame.from_dict(config)
    sc = mb.get_scenario(name=scen_name)
    if sc is None:
        sc = mb.get_scenario(name='Scenario').copy(name=scen_name)
    curr_dt = '14/06/2018'
    curr_dt_obj = datetime.strptime(curr_dt, '%d/%m/%Y')
    for it in xrange(1, (its + 1)):
        print("Solving {} -- day {}".format(scen_name, it))
        for table_name in table_list:
            if sc.get_asset(table_name):
                sc.delete_asset(table_name)
            df = input_dict[table_name]
            if table_name in ['df_thermal_units_conf', 'df_renewable_units_conf', 'df_load', 'df_scenarios', 'df_renewable_forecast', 'df_renewable_forecast_aggr']:
                sc.add_table_data(table_name, extract_df_by_period(df, DateTime(curr_dt, '1'), uc_days), category='input')
            else:
                sc.add_table_data(table_name, df, category='input')
        
        
        sc.add_table_data('config', df_config, category='input')
        
        sc.solve(display_log=True)
        df_q_r_tot = sc.get_table_data('p_r_tot').copy()
        df_q_r_tot['day'] = "{:0>2d}".format(it)
        df_q_r_tot['model'] = model
        df_p_tot_by_period = sc.get_table_data('thermal_prod').groupby('period').sum().copy()
        df_p_tot_by_period.reset_index(inplace=True)
        df_p_tot_by_period.rename(columns={'value':'thermal_prod'}, inplace=True)
        df_p_inde_tot_by_period = sc.get_table_data('renewable_prod').groupby('period').sum().copy()
        df_p_inde_tot_by_period.reset_index(inplace=True)
        df_p_inde_tot_by_period.rename(columns={'value':'renewable_prod'}, inplace=True)
        df_p_tot_by_period = df_p_tot_by_period.merge(df_p_inde_tot_by_period, on='period', how='inner')
        df_q_r_tot = df_q_r_tot.merge(df_p_tot_by_period, on='period', how='inner')
        
        df_cost = sc.get_table_data('cost_kpis_high').copy()
        df_cost.set_index('kpi', inplace=True)
        display(df_cost)
        tot_cost = df_cost.at['Total Economic Cost', 'value']
        df_q_r_tot['cost'] = tot_cost

        if df_final.empty:
            df_final = df_q_r_tot
        else:
            df_final = pd.concat([df_final, df_q_r_tot], ignore_index=True)

        curr_dt_obj = curr_dt_obj + timedelta(days=uc_days)
        curr_dt = curr_dt_obj.date().strftime('%d/%m/%Y')

sc = mb.get_scenario(name="Scenario")

sc.add_table_data('scenario_comp', df_final, category='input')
df_final.to_csv("{}/datasets/df_dashboard.csv".format(os.environ['DSX_PROJECT_DIR']), index=False)

Solving Scen-D -- day 1
[2018-10-02T14:08:48Z, INFO] , solutions = 5
[2018-10-02T14:08:48Z, INFO] )
[2018-10-02T14:08:49Z, INFO] *   480+  181                       2.32717e+07   2.32016e+07             0.30%
[2018-10-02T14:08:50Z, INFO]     500   195   2.32351e+07   314   2.32717e+07   2.32018e+07    34123    0.30%
[2018-10-02T14:08:51Z, INFO]     563   240   2.32505e+07   289   2.32717e+07   2.32044e+07    40663    0.29%
[2018-10-02T14:08:53Z, INFO] *   650+  114                       2.32274e+07   2.32050e+07             0.10%
[2018-10-02T14:08:53Z, INFO] *   680+  139                       2.32244e+07   2.32051e+07             0.08%
[2018-10-02T14:08:53Z, INFO] *   680+  138                       2.32236e+07   2.32051e+07             0.08%
[2018-10-02T14:08:53Z, INFO]     680   140   2.32175e+07    96   2.32236e+07   2.32051e+07    47304    0.08%
[2018-10-02T14:08:55Z, INFO] *   780+  152                       2.32189e+07   2.32065e+07             0.05%
[2018-10-02T14:08:55Z, INFO]

,value
kpi,
Total Variable Cost,23133470.0
Total Economic Cost,23192868.0


Solving Scen-D -- day 2
[2018-10-02T14:11:02Z, INFO]    4919  1256        cutoff         2.33413e+07   2.33299e+07   270880    0.05%
[2018-10-02T14:11:05Z, INFO]    4989  1207   2.33426e+07   225   2.33413e+07   2.33305e+07   275422    0.05%
[2018-10-02T14:11:08Z, INFO]    5091  1159   2.33404e+07   224   2.33413e+07   2.33312e+07   280564    0.04%
[2018-10-02T14:11:08Z, INFO] Elapsed time = 117.59 sec. (64733.90 ticks, tree = 23.46 MB
[2018-10-02T14:11:08Z, INFO] , solutions = 8
[2018-10-02T14:11:08Z, INFO] )
[2018-10-02T14:11:11Z, INFO]    5179  1101   2.33371e+07   249   2.33413e+07   2.33320e+07   285015    0.04%
[2018-10-02T14:11:13Z, INFO]    5278  1031        cutoff         2.33413e+07   2.33330e+07   289613    0.04%
[2018-10-02T14:11:16Z, INFO]    5371   980        cutoff         2.33413e+07   2.33335e+07   293781    0.03%
[2018-10-02T14:11:20Z, INFO]    5522   897        cutoff         2.33413e+07   2.33344e+07   299058    0.03%
[2018-10-02T14:11:25Z, INFO]    5713   764   2.3

,value
kpi,
Total Variable Cost,23252560.0
Total Economic Cost,23319288.0


Solving Scen-D -- day 3
[2018-10-02T14:12:10Z, INFO] *   827+  143                       2.36199e+07   2.35813e+07             0.16%
[2018-10-02T14:12:12Z, INFO]     909   201   2.35974e+07   296   2.36199e+07   2.35850e+07    51254    0.15%
[2018-10-02T14:12:14Z, INFO]     979   248   2.35963e+07   288   2.36199e+07   2.35859e+07    55453    0.14%
[2018-10-02T14:12:14Z, INFO] Elapsed time = 26.44 sec. (16069.87 ticks, tree = 7.63 MB
[2018-10-02T14:12:14Z, INFO] , solutions = 11
[2018-10-02T14:12:14Z, INFO] )
[2018-10-02T14:12:15Z, INFO]    1080   322   2.36072e+07   215   2.36199e+07   2.35871e+07    59508    0.14%
[2018-10-02T14:12:17Z, INFO]    1217   432   2.36066e+07   234   2.36199e+07   2.35885e+07    64487    0.13%
[2018-10-02T14:12:18Z, INFO] *  1230   316      integral     0   2.36121e+07   2.35885e+07    64839    0.10%
[2018-10-02T14:12:18Z, INFO] *  1260   196      integral     0   2.36042e+07   2.35885e+07    65623    0.07%
[2018-10-02T14:12:20Z, INFO]    1344   211   2.35

...

,value
kpi,
Total Variable Cost,23354287.0
Total Economic Cost,23388616.0


Solving Scen-S -- day 6
[2018-10-02T14:24:04Z, INFO]       0     0   2.40863e+07  2029                  Cuts: 1951     7889
[2018-10-02T14:24:05Z, INFO]       0     0   2.40873e+07  1010                  Cuts: 1951     9013
[2018-10-02T14:24:05Z, INFO]       0     0   2.40909e+07   164                  Cuts: 1951    10144
[2018-10-02T14:24:05Z, INFO]       0     0   2.40950e+07   162                   Cuts: 389    10406
[2018-10-02T14:24:06Z, INFO]       0     0   2.40986e+07   176                   Cuts: 250    10614
[2018-10-02T14:24:06Z, INFO]       0     0   2.41001e+07   160                   Cuts: 250    10835
[2018-10-02T14:24:06Z, INFO]       0     0   2.41006e+07   183                   Cuts: 188    10972
[2018-10-02T14:24:06Z, INFO]       0     0   2.41014e+07   135                   Cuts: 198    11109
[2018-10-02T14:24:06Z, INFO]       0     0   2.41015e+07   189                   Cuts: 141    11244
[2018-10-02T14:24:08Z, INFO] *     0+    0                       2.41088e+07

,value
kpi,
Total Variable Cost,23779622.0
Total Economic Cost,23807628.0


Solving Scen-S -- day 7
[2018-10-02T14:24:39Z, INFO]    Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap
[2018-10-02T14:24:39Z, INFO] *     0+    0                       2.32395e+07   2.32358e+07             0.02%
[2018-10-02T14:24:40Z, INFO]       0     0   2.32358e+07  1568   2.32395e+07   2.32358e+07    14357    0.02%
[2018-10-02T14:24:40Z, INFO]       0     0   2.32358e+07  1384   2.32395e+07     Cuts: 892    14517    0.02%
[2018-10-02T14:24:40Z, INFO]       0     0   2.32358e+07  1095   2.32395e+07     Cuts: 892    14808    0.02%
[2018-10-02T14:24:40Z, INFO]       0     0   2.32358e+07   480   2.32395e+07     Cuts: 892    15425    0.02%
[2018-10-02T14:24:41Z, INFO]       0     0   2.32358e+07   129   2.32395e+07     Cuts: 892    16013    0.02%
[2018-10-02T14:24:41Z, INFO]       0     0   2.32359e+07    30   2.32395e+07     Cuts: 131    16106    0.02%
[2018-10-02T14:24:41Z, INFO]       0     0   2.32360e+07    26   2.32395e+07      Cuts: 21    16118    0.

,value
kpi,
Total Variable Cost,22894320.0
Total Economic Cost,22923097.0


### Author

- __Gianmaria Leo__ is a Senior Operations Research Engineer and Data Scientist with Data Science Elite team (IBM Analytics).

Copyright © IBM Corp. 2018. Released as licensed Sample Materials.